# Setup Guide

This guide outlines the steps to set up and run the AI-powered marketing analysis tool demonstrated in this notebook.

1. Install Libraries:
  - OpenAI
  - Swarm
2. Import Libraries
3. Authenticate with OpenAI:
  - Replace `''` with your actual OpenAI API key
4. Initialize Swarm Client
5. Load Data
6. Define Agents:
  - **Performance Evaluator:** Evaluates traffic sources and campaigns based on key metrics.
  - **Trend Analyzer:** Identifies patterns and anomalies in campaign performance over time.
  - **Campaign Recommender:** Provides recommendations for optimizing campaigns and budget allocation.

7. Run the Analysis:*
  - Execute the code cells to run the agents and generate insights.

8. Review Results:*
  - The notebook outputs performance evaluations, trend analysis, and campaign recommendations.

**Note:** Make sure you have an active internet connection to access the data and OpenAI API.

In [1]:
!pip install git+https://github.com/openai/swarm.git
!pip install openai httpx==0.27.2

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-rllld51c
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-rllld51c
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import pandas as pd
import numpy as np
from openai import OpenAI
from swarm import Swarm, Agent
from scipy.stats import zscore

In [3]:
api = OpenAI(api_key='')

In [4]:
client = Swarm(api)

In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/cherrymaegrace/AIR_Projects/refs/heads/main/06_optimized_marketing_ai_agents/dataset/ai-first-sales-data.csv')
data.head()

,date,source,medium,delivery_available,device_type,promo_activated,filter_used,pageviews,visits,productClick,addToCart,checkout,transactions,revenue,ad spend
0,2020-05-11,google,organic,no data,PC,no,no,4087,1233,5240,1048,672.00,90,"₱456,877","₱384,039"
1,2020-05-11,facebook,cpc,no data,mobile,yes,no,4326,544,9930,1984,1812.48,217,"₱1,289,066","₱817,514"
2,2020-05-11,google,cpc,no data,mobile,no,no,3891,1450,5460,1090,766.72,100,"₱554,427","₱435,105"
3,2020-05-11,google,cpc,no data,PC,no,no,2456,854,4250,848,520.96,71,"₱416,561","₱635,599"
4,2020-05-11,facebook,organic,no data,PC,no,no,2828,1000,4110,824,449.28,62,"₱326,176","₱428,962"


In [6]:
struct = []

# Data Processing Functions

In [7]:
# Cleaning and formatting the dataset
def clean_data(context_variables):
  data = context_variables['data']

  # Remove any commas and currency symbols from 'revenue' and 'ad spend' columns and convert to numeric
  data['revenue'] = data['revenue'].str.replace('[₱, ]', '', regex=True).astype(float)
  data['ad spend'] = data['ad spend'].str.replace('[₱, ]', '', regex=True).astype(float)

  # Check for missing or inconsistent data
  missing_data_summary = data.isnull().sum()

  # Ensure all column types are appropriate and consistent
  data_types = data.dtypes

  data_info = {
      'first_five_rows': data.head(),
      'missing_data_summary': missing_data_summary,
      'data_types': data_types
  }

  return data_info

In [8]:
# Computing performance metrics
def compute_metrics(data):
  data['transactions'] = data['transactions'].replace(0, np.nan)
  data['addToCart'] = data['addToCart'].replace(0, np.nan)

  # Conversion Rate (CR)
  data['conversion_rate'] = data['transactions'] / data['visits']

  # Return on Ad Spend (ROAS)
  data['roas'] = data['revenue'] / data['ad spend']

  # Cost per Acquisition (CPA)
  data['cpa'] = data['ad spend'] / data['transactions']

  # Engagement Rate (ER)
  data['engagement_rate'] = data['productClick'] / data['pageviews']

  # Cart Abandonment Rate (CAR)
  data['cart_abandonment_rate'] = (data['addToCart'] - data['checkout']) / data['addToCart']

  # Revenue per Visit (RPV)
  data['revenue_per_visit'] = data['revenue'] / data['visits']

  return data

In [9]:
# Aggregate metrics by source and medium
def aggregate_metrics(data):

  aggregated_metrics = data.groupby(['source', 'medium']).agg({
      'conversion_rate': 'mean',
      'roas': 'mean',
      'cpa': 'mean',
      'engagement_rate': 'mean',
      'cart_abandonment_rate': 'mean',
      'revenue_per_visit': 'mean'
  }).reset_index()

  return aggregated_metrics

In [10]:
# Performance evaluation: Ranking sources and flagging improvements
def evaluate_performance(context_variables):
  data = context_variables['data']

  # Calculate metrics
  data = compute_metrics(data)

  # Aggregate metrics by source and medium
  aggregated_metrics = aggregate_metrics(data)

  # Rank sources by ROAS, Conversion Rate, and Revenue per Visit
  ranked_metrics = aggregated_metrics.sort_values(by=['roas', 'conversion_rate', 'revenue_per_visit'], ascending=False)

  # Flag sources with high CPA or CAR as areas needing improvement
  threshold_cpa = data['cpa'].mean()  # Using the mean CPA as a threshold for "high"
  threshold_car = data['cart_abandonment_rate'].mean()  # Using the mean CAR as a threshold for "high"

  # Adding flags for underperforming metrics
  ranked_metrics['high_cpa_flag'] = ranked_metrics['cpa'] > threshold_cpa
  ranked_metrics['high_car_flag'] = ranked_metrics['cart_abandonment_rate'] > threshold_car

  return ranked_metrics[['source', 'medium', 'roas', 'conversion_rate', 'revenue_per_visit']]

In [11]:
def analyze_trends(context_variables):
  data=context_variables['data']

  # Calculate metrics
  data = compute_metrics(data)

  # Aggregate metrics by date
  daily_metrics = data.groupby('date').agg({
      'conversion_rate': 'mean',
      'roas': 'mean',
      'cpa': 'mean'
  }).reset_index()

  # Detect anomalies using z-scores
  daily_metrics['roas_zscore'] = zscore(daily_metrics['roas'])

  # Identify significant anomalies (z-score threshold)
  anomalies = daily_metrics[daily_metrics['roas_zscore'].abs() > 2]

  anomalies_detected = f'Anomalies detected: \n\n {anomalies}'

  return anomalies_detected

# Agents

## **Agent: Performance Evaluator**
**Description**:  
This agent evaluates the effectiveness of traffic sources and campaigns by ranking them based on key performance metrics. It flags underperforming areas to identify opportunities for improvement.

**Analysis Methodology**:
1. **Aggregate Metrics**:
   - Group data by traffic source and medium.
   - Calculate averages for metrics like ROAS, CR, CPA, etc.
2. **Rank Campaigns**:
   - Sort by primary metrics (e.g., ROAS, CR) to identify top performers.
3. **Flag Underperformers**:
   - Define thresholds (e.g., mean CPA or CAR) and flag campaigns exceeding them.
4. **Output Insights**:
   - Provide a ranked list of campaigns and highlight areas needing improvement.

In [12]:
performance_evaluator = Agent(
    name="Performance Evaluator",
    instructions="Identify and rank the best-performing traffic sources and campaigns using metrics like ROAS, CR, and RPV. Highlight sources with high CPA or CAR for improvement.",
    model="gpt-4o-mini",
    functions=[clean_data, evaluate_performance]
)

## **Agent: Trend Analyzer**
**Description**:  
This agent identifies patterns and anomalies in campaign performance over time, providing insights into seasonality and sudden changes in effectiveness.

**Analysis Methodology**:
1. **Aggregate Data by Time**:
   - Group metrics by date, week, or month to analyze time trends.
2. **Detect Anomalies**:
   - Use statistical methods (e.g., z-scores, interquartile range) to identify outliers.
3. **Seasonal Analysis**:
   - Highlight periodic peaks or dips aligning with promotions or holidays.
4. **Output Insights**:
   - Summarize significant trends and suggest data-driven responses.

In [13]:
trend_analyzer = Agent(
    name="Trend Analyzer",
    instructions="Analyze trends over time for metrics such as ROAS, CR, and RPV. Detect and flag anomalies or sudden changes in performance.",
    model="gpt-4o-mini",
    functions=[clean_data, analyze_trends]
)

## **Agent: Campaign Recommender**
**Description**:  
This agent leverages insights from performance and trend analysis to provide actionable recommendations for optimizing campaigns and reallocating budgets.

**Analysis Methodology**:
1. **Incorporate Insights**:
   - Use flagged metrics (e.g., high CPA or CAR) and trend anomalies as inputs.
2. **Recommend Optimizations**:
   - Suggest targeting changes, cost adjustments, or content improvements for underperforming campaigns.
3. **Budget Allocation**:
   - Propose reallocating budgets to maximize ROAS and overall performance.
4. **Highlight Opportunities**:
   - Identify campaigns with growth potential during peak periods or for specific audience segments.
5. **Output Recommendations**:
   - Provide a prioritized list of actionable strategies for campaign improvements.

In [14]:
campaign_recommender = Agent(
    name="Campaign Recommender",
    model="gpt-4o-mini",
    instructions="Based on the analysis of traffic sources, campaigns, and performance trends, recommend strategies to optimize underperforming campaigns. Suggest reallocation of budget to maximize ROAS and improve revenue generation. Address flagged issues such as high CPA or high CAR with actionable improvements.",
)

# **Implementation**

In [15]:
def to_campaign_recommender():
  return campaign_recommender

agent = Agent(
    functions=[to_campaign_recommender]
)

In [16]:
performance_evaluator_response = client.run(
    agent = performance_evaluator,
    messages = struct,
    context_variables = {'data': data}
)
perf_eval = performance_evaluator_response.messages[-1]['content']

In [17]:
trend_analyzer_response = client.run(
    agent = trend_analyzer,
    messages = struct,
    context_variables = {'data': data}
)
trend_analysis = trend_analyzer_response.messages[-1]['content']

In [18]:
struct.append({'role':'assistant', 'content': f'Performance Evaluation:\n\n {perf_eval}'})
struct.append({'role':'assistant', 'content': f'Trend Analysis:\n\n {trend_analysis}'})

In [19]:
campaign_reco_response = client.run(
    agent = agent,
    messages = struct,
    context_variables = {'data': data}
)

campaign_recommendations = campaign_reco_response.messages[-1]['content']

# Findings and Reports

## Performance Evaluation

In [21]:
print(perf_eval)

### Performance Analysis of Traffic Sources and Campaigns

**Best-Performing Traffic Sources:**

Here’s a ranking of the top traffic sources based on Return on Ad Spend (ROAS), Conversion Rate (CR), and Revenue per Visit (RPV):

| Rank | Source         | Medium   | ROAS    | Conversion Rate | Revenue per Visit |
|------|----------------|----------|---------|------------------|--------------------|
| 1    | Google         | Organic  | 0.6507  | 0.5322           | 2866.12            |
| 2    | Facebook       | CPC      | 0.6000  | 0.5127           | 2936.31            |
| 3    | Facebook       | Organic  | 0.4557  | 0.5164           | 2832.68            |
| 4    | Google         | CPC      | 0.4005  | 0.5032           | 2876.96            |
| 5    | TikTok         | CPA      | 0.2088  | 0.5112           | 2504.06            |
| 6    | Cityads        | CPA      | 0.0641  | 0.5073           | 2350.13            |
| 7    | Instagram      | CPC      | 0.0610  | 0.5316           | 2487.15    

## Trend Analysis

In [22]:
print(trend_analysis)

### Data Cleaning Results
The dataset was successfully cleaned and contains the following information:
- **First Five Rows**: Sample data that includes various metrics such as `date`, `source`, `medium`, `pageviews`, `visits`, `transactions`, `revenue`, and `ad spend`.
- **Missing Data Summary**: No missing values were found in any of the columns.
- **Data Types**: Each column has been correctly identified with appropriate data types (e.g., integers for counts, floats for revenue and ad spend, and objects for categorical variables).

### Trends Analysis Results
During the analysis of performance metrics over time, several anomalies were detected. Here are some key findings:

- **Anomalies Detected**: Specific dates with unusual metrics were flagged:
  ```
                date  conversion_rate      roas            cpa  roas_zscore
  44   2020-02-14         0.496201  0.655886  209378.943429     3.416999
  45   2020-02-15         0.581179  0.626678  190863.324569     3.110732
  50   2020-

## Campaign Recommendation

In [23]:
print(campaign_recommendations)

### Campaign Optimization Recommendations

Based on the analysis of traffic sources, campaigns, and performance trends, here are actionable strategies to optimize underperforming campaigns, reallocate budgets, and address flagged issues:

### 1. Reallocate Budget

**Reduce Spend on Underperforming Sources:**
- **Actionpay, Other, Mytarget:** These sources have high CPA and low ROAS, indicating poor performance. Temporarily reduce or halt budget allocation to these campaigns.
- **Recommendation:** Allocate a greater share of the marketing budget to high-performing channels like **Google Organic** and **Facebook CPC**, which have shown strong ROAS and conversion rates. A suggested split could be:
  - **Google Organic**: Increase budget from 20% to 30%
  - **Facebook CPC**: Increase budget from 15% to 25%
  - **Other Sources**: Reduce total budget allocation to 15% (from 40% combined)

### 2. Improve Underperforming Campaigns

**Targeted Improvements for High CPA Campaigns:**
- **Actionpa